In [1]:
import requests
from transformers import pipeline

In [3]:
!pip install --upgrade requests

Requirement already up-to-date: requests in c:\users\kamik\anaconda3\envs\learn-env\lib\site-packages (2.24.0)


In [2]:
nlp = pipeline("sentiment-analysis")

In [5]:
result = nlp("Lovely atmosphere, staff are super friendly and wonderful people.")[0]

In [6]:
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9999
